* merge_variants 결과에 심대한 오류 (간체 텍스트 경우) 있음

In [92]:
import re
from hanziminer import *

In [93]:
# AUTO RELOAD : https://goo.gl/hYKqVS
%load_ext autoreload
%autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## TEST

In [30]:
text = open("_tmp/_dummy_formulas.yml", 'r', encoding="utf-8").read()
corpus = Corpus( text[10000:20000] )
corpus.merge_duplications().merge_variants()
corpus.remove_comments( comments_header="//" ).remove_punctuation().remove_chrs().merge_spaces()
corpus.export("_tmp/_dummy_formulas_clean.txt", plain_text=True )
# corpus.export("_tmp/_dummy_formulas_docs.txt", plain_text=False )

# Duplicated Characters were merged
# Variants Characters were merged
# Comments were removed
# Punctuations were removed
# Korean, Alphabet, Numbers were removed
# Spaces were merged
# File _tmp/_dummy_formulas_clean.txt was Created


In [31]:
te = TokenExtractor( corpus )
te.train().extract().report("_tmp/_dummy_formulas_tokens.txt")

# Training ... 
Progress |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
# Training was done. Used memory 0.100 Gb
# Extracting ...
Progress |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
# Extrating was done. System memory 0.100 Gb used
# 144 of tokens were reported in _tmp/_dummy_formulas_tokens.txt


In [58]:
sg = SegmenterScore( te.score(), method='cohesion' )
input_doc = re.sub( r"(\r?\n){2}", "\n", corpus.to_text() )
output_doc = open("_tmp/_dummy_formulas_tokenized.txt", 'w', encoding="utf-8")
for line in input_doc.split("\n"):
    if line.strip() == "":
        output_doc.write("\n")
    else:
        output_doc.write( " ".join( sg.load( line ).segment().to_list( verbose=False, keyword_only=False ) ) )
        output_doc.write("\n")
output_doc.close()
print("Completed")    

Completed


In [59]:
sg.load( line ).segment().to_list( verbose=False, keyword_only=False )

['肺郁痰火或陰虛肺熱所致之咳嗽']

In [60]:
sg2 = SegmenterGram( gram_size=2 )
output_doc = open("_tmp/_dummy_formulas_tokenized2.txt", 'w', encoding="utf-8")
for line in input_doc.split("\n"):
    if line.strip() == "":
        output_doc.write("\n")
    else:
        output_doc.write( sg2.load( line ).segment().to_string() )
        output_doc.write("\n")
output_doc.close()
print("Completed")    

Completed


In [61]:
txt = open("_tmp/_dummy_formulas_tokenized.txt", 'r', encoding="utf-8").read()
cq = COQuantifier().load( txt  ).count().score()
cq.export( "_tmp/_dummy_formulas_당귀.txt", "當歸", cutoff=0.1 )

# Co-occurrence Counting ... 
Progress |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
# Co-occurrence Counting was done. System memory 0.091 Gb used
# All Tokens were Counted
# Co-occurrence Scores Generating ... 
Progress |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
# Co-occurrence Scores were generated. System memory 0.091 Gb used


In [63]:
txt2 = open("_tmp/_dummy_formulas_tokenized2.txt", 'r', encoding="utf-8").read()
cq2 = COQuantifier().load( txt2  ).count().score()
cq2.export( "_tmp/_dummy_formulas_당귀2.txt", "當歸", cutoff=0.1 )

# Co-occurrence Counting ... 
Progress |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
# Co-occurrence Counting was done. System memory 0.091 Gb used
# All Tokens were Counted
# Co-occurrence Scores Generating ... 
Progress |████████████████████████████████████████████████████████████████████████████████████████████████████| 100.0% Complete
# Co-occurrence Scores were generated. System memory 0.091 Gb used
# Report File was exported 


In [ ]:
ss = SegmenterDict( ["a", "ab", "abc", "bc", "bcd", "cdef", "fgh"] )
ss.load("abcdefghijkl").segment().to_list()

In [ ]:
sg = Segmenter( te.score(), target_score='cohesion' )
# sgl = Segmenter( te.score, target_score='cohesion_l' )
# sgr = Segmenter( te.score, target_score='cohesion_r' )
sgE = Segmenter( te.score(), target_score='branch_entropy' )


docs = [
    "治風證眩暈. 山茱萸肉 一兩, 山藥ㆍ甘菊ㆍ人參ㆍ川芎ㆍ茯神 各五錢. 右爲末, 每二錢, 酒調下. 《本事》",
    "眞人養生銘曰人欲勞於形百病不能成飮酒勿大醉諸疾自不生食了行百步數以手摩肚寅丑日剪甲頭髮梳百度飽卽立小便飢則坐漩尿行處勿當風居止無小隙常夜濯足臥飽食終無益思慮最傷神喜怒最傷氣每去鼻中毛常習不唾地平明欲起時下床先左脚一日無災殃去邪兼辟惡如能七星步令人長壽樂酸味傷於筋苦味傷於骨甘卽不益肉辛多敗正氣鹹多促人壽不得偏耽嗜春夏少施泄秋冬固陽事獨臥是守眞愼靜最爲貴錢財生有分知足將爲利强知是大患少慾終無累神靜自常安修道宜終始書之屋壁中將以傳君子",
    "久服明目輕身延年酒浸曝乾蒸之如此九次搗爲末每二錢空心溫酒調服一日二次本草",
    "治折傷後爲風寒濕所侵手足疼痛生蒼朮破古紙半生半炒骨碎補穿山甲桑灰炒爲珠生草烏各二兩茴香一兩半右將草烏剉如麥大同連皮生薑四兩擂爛淹兩宿焙乾同前藥爲末酒糊和丸梧子大溫酒下五十丸少麻無妨得效"
]

for sn in docs:
    sg.load( sn ).segment()
    #sgl.load( sn ).segment()
    #sgr.load( sn ).segment()
    sgE.load( sn ).segment()
#     print( sgs.token_candis )
    #print( sg.target_text )
    print( sg.to_string( verbose=True, keyword_only=True ) ) # not working
    print( sg.to_string( verbose=True, keyword_only=False ) )
    print( sg.to_string( verbose=False, keyword_only=True ) ) 
    print( sg.to_string( verbose=False, keyword_only=False ) )
    
    print( sg.to_list( verbose=True, keyword_only=True ) )
    print( sg.to_list( verbose=True, keyword_only=False ) ) # not working
    print( sg.to_list( verbose=False, keyword_only=True ) )
    print( sg.to_list( verbose=False, keyword_only=False ) )
    
    #print( sgl.show() )
    #print( sgr.show() )
#     print( sgE.show() )
    print("\n")


## SANDBOX

In [ ]:
np.power( 0.7, 1/2 )

In [ ]:
print(te.__file__)

In [ ]:
rst = sorted( te.score.items(), key=lambda it: -1* ( it[1].cohesion_r * it[1].cohesion_l ) )
tmp2 = open("_tmp/_dummy_corpus_score.txt", 'w', encoding="utf-8")
pp = pprint.PrettyPrinter(indent=4, stream=tmp2)
pp.pprint( [ ( r[0], r[1].freq , r[1].cohesion_r, r[1].cohesion_l ) for r in rst ] )

print("ending")

In [ ]:
te.unigram_counter.get("枸")

In [ ]:
from collections import Counter
c = Counter("huihihu ihuihi") 
d = Counter("werwwqweqriiiiiiiiiiii")
c.update(d)
print(c)

In [ ]:
97 / 8

In [ ]:
"c" in {'a':1, 'b':2}.keys()


In [ ]:
dict( [('a', 1), ('b', 2)] )

In [ ]:
import math
def entropy( p ):
    return -1 * p * math.log2( p )
entropy( 100 )

In [ ]:
"123456789"[1:]

In [ ]:


import os
import psutil
import sys
from collections import defaultdict

def get_available_memory():
    """It returns remained memory as percentage"""

    mem = psutil.virtual_memory()
    return 100 * mem.available / (mem.total)

def get_process_memory():
    """It returns the memory usage of current process"""
    
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 ** 3)


sys.stdout.write('\rtraining ... (%d in %d sents) use memory %.3f Gb' % (100, 100, get_process_memory()))
print('\rtraining ... (%d in %d sents) use memory %.3f Gb' % (100, 100, get_process_memory()))
print('\rtraining was done. used memory %.3f Gb' % (get_process_memory()))

In [ ]:
import re
rg = re.compile("{0}\d+?{0}".format("%") )
# rg = re.compile("\d+?" )
txt = "朮破古紙半%1%生半炒骨碎%12%補穿山甲桑%8%灰炒爲珠生"
print( re.findall( rg, txt) )

In [ ]:
for x in Counter(['a', 'a', 'b']):
    print(x)

from itertools import chain, combinations
k = Counter(['a', 'a', 'b', 'c'])
combinations( k.keys(), 2) 


In [ ]:
k = ['\\', '|', '/', '-']
for i in range(0, 1000):
    sys.stdout.write("\r# {}".format( k[i % 4]) )
    
    

In [ ]:
import re
if not re.search( re.compile("[ \t]+"), "a1aa1aaa1aaaa1aaaa1" ): print("efef")

In [ ]:
%load_ext autoreload

In [68]:
k = [0] * 15
k[4:6] = [1]*2
k

[0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [94]:
ss = SegmenterDict( ["a", "ab", "abc", "bc", "bcd", "cdef", "fgh"] )
ss.load("abcdefghijkl").segment().to_list()


['', 'abc', 'de', 'fgh', 'ijkl']

In [95]:
r"wer"

'wer'